In [142]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
from pygeocoder import Geocoder
import reverse_geocoder as rg 
import pprint 
from folium.plugins import MarkerCluster

from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from uszipcode import Zipcode

In [143]:
df = pd.read_csv("crime.csv")
df.head()

,Dc_Dist,Psa,Dispatch_Date_Time,Dispatch_Date,Dispatch_Time,Hour,Dc_Key,Location_Block,UCR_General,Text_General_Code,Police_Districts,Month,Lon,Lat
0,18,3,2009-10-02 14:24:00,2009-10-02,14:24:00,14,200918067518,S 38TH ST / MARKETUT ST,800.0,Other Assaults,NaN,2009-10,NaN,NaN
1,14,1,2009-05-10 00:55:00,2009-05-10,00:55:00,0,200914033994,8500 BLOCK MITCH,2600.0,All Other Offenses,NaN,2009-05,NaN,NaN
2,25,J,2009-08-07 15:40:00,2009-08-07,15:40:00,15,200925083199,6TH CAMBRIA,800.0,Other Assaults,NaN,2009-08,NaN,NaN
3,35,D,2009-07-19 01:09:00,2009-07-19,01:09:00,1,200935061008,5500 BLOCK N 5TH ST,1500.0,Weapon Violations,20.0,2009-07,-75.130477,40.036389
4,9,R,2009-06-25 00:14:00,2009-06-25,00:14:00,0,200909030511,1800 BLOCK WYLIE ST,2600.0,All Other Offenses,8.0,2009-06,-75.166350,39.969532


In [144]:
df.dropna(subset = ["Lon"], inplace=True)
df.dropna(subset = ["Lat"], inplace=True)
df['Dispatch_Date'] = pd.to_datetime(df['Dispatch_Date'], errors='coerce')
include = df[(df.Dispatch_Date.dt.year == 2016) & (df.Dispatch_Date.dt.month == 5)]

In [145]:
location=include.groupby(['Lat','Lon','Text_General_Code']).size().reset_index(name='cnt').sort_values(by='cnt',ascending=False)
location.sort_values('cnt')
location.head()


,Lat,Lon,Text_General_Code,cnt,combined
8857,40.022416,-75.078182,All Other Offenses,53,"(40.022416, -75.078182)"
3387,39.960800,-75.224767,All Other Offenses,42,"(39.9608, -75.224767)"
9438,40.028490,-75.099584,Thefts,31,"(40.028490000000005, -75.099584)"
10441,40.039082,-75.144752,Fraud,23,"(40.039082, -75.144752)"
6777,39.995882,-75.132840,All Other Offenses,21,"(39.995882, -75.13284)"


In [146]:
def get_zipcode(lat, lon):
    result = search.by_coordinates(lat = lat, lng = lon, returns = 1)
    return result[0].zipcode

location['zipcode'] = location.apply(lambda x: get_zipcode(x.Lat,x.Lon), axis=1)


In [148]:

location['color']=location['cnt'].apply(lambda count:"black" if count==53 else
                                        "grey" if count==42 else
                                        "red" if count==31 else
                                        "violet" if count==23 else
                                        "indianred" if count==21 else
                                        "red" if count==20 else
                                        "deeppink" if count==17 else
                                        "purple" if count==16 else
                                        "darkviolet" if count==15 else
                                        "slateblue" if count==14 else
                                        "green" if count==13 else
                                        "lightgreen" if count==12 else
                                        "aqua" if count==11 else
                                        "dodgerblue" if count==10 else
                                        "darkorange" if count==9 else
                                        "orange" if count==8 else
                                        "forestgreen" if count==7 else
                                        "limegreen" if count==6 else
                                        "rawgreen" if count==5 else
                                        "darkcyan" if count==4 else
                                        "blue" if count==3 else
                                        "red" if count==2 else
                                         "green")

location['size']=location['cnt'].apply(lambda count:22 if count==53 else
                                        21 if count==42 else
                                        19 if count==31 else
                                        17 if count==23 else
                                        16 if count==21 else
                                        16 if count==20 else
                                        14 if count==17 else
                                        13 if count==16 else
                                        12 if count==15 else
                                        12 if count==14 else
                                        10 if count==13 else
                                        10 if count==12 else
                                        10 if count==11 else
                                        9 if count==10 else
                                        9 if count==9 else
                                        9 if count==8 else
                                        8 if count==7 else
                                        6 if count==6 else
                                        5 if count==5 else
                                        4 if count==4 else
                                        3 if count==3 else
                                        3 if count==2 else
                                        0.7)




In [162]:
m=folium.Map([39.9527237,-75.1635262],zoom_start=11.4)
for lat,lon,code,zipcode,color,count,size in zip(location['Lat'],location['Lon'],location['Text_General_Code'],location['zipcode'],location['color'],location['cnt'],location['size']):
    folium.CircleMarker([lat, lon],
                            popup = folium.Popup((
    "Type: {type}<br>"
    "Zip: {zip}<br>"
   ).format(type=code,
            zip=zipcode),max_width=265),
                            radius=size,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color=color,
                           ).add_to(m)

m.save('PHL_map.html')





In [163]:
location_data = location[['Lat', 'Lon']].values

# plot heatmap
m.add_child(plugins.HeatMap(location_data, radius=15))
m.save('PHL_heatmap.html')

In [168]:
location_df=location

some_map_2 = folium.Map(location=[location_df['Lat'].mean(),location_df['Lon'].mean()],zoom_start=10)

mc = MarkerCluster()

for row in location_df.itertuples():
    my_string = "Type : "+row.Text_General_Code+"\nZip : "+row.zipcode
    mc.add_child(folium.Marker(location=[row.Lat,row.Lon],popup = folium.Popup((
    "Type: {type}<br>"
    "Zip: {zip}<br>"
   ).format(type=row.Text_General_Code,
            zip=row.zipcode),max_width=265)))

some_map_2.add_child(mc)
some_map_2.save('PHLMasterCluster.html')